In [1]:
import nilearn as nl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle as pkl
import time
import os
import math
from nilearn import plotting
from nilearn import datasets
import time

### Data Fetch & Parameters Setting

In [2]:
data = pkl.load(open( "../data/shuffled_data_pcp.p", "rb" ))
trn_x = data['trn_x']
trn_y = data['trn_y']
val_x = data['val_x']
val_y = data['val_y']

num_trn = trn_x.shape[0]
num_val = val_x.shape[0]
num_node = 48

n_trn_y = np.zeros((num_trn, 1))
n_val_y = np.zeros((num_val, 1))

for i in range(num_trn):
    if trn_y[i][0] == 1:
        n_trn_y[i] = 0
    else:
        n_trn_y[i] = 1

for i in range(num_val):
    if val_y[i][0] == 1:
        n_val_y[i] = 0
    else:
        n_val_y[i] = 1

trn_y = n_trn_y
val_y = n_val_y

num_label = trn_y.shape[1]
print(num_label, num_trn, num_val)
print(trn_y.shape, val_y.shape)
print(trn_y, val_y)

1 696 175
(696, 1) (175, 1)
[[1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]


In [3]:
#Gaussian Kernel
sigma = 1
def similarity (a, b):
    temp = a - b;
    sim = np.exp(-np.dot(temp, temp)/(2 * np.square(sigma)));
    return sim

landmarks = trn_x

n_trn_x = np.zeros((num_trn, num_trn))
n_val_x = np.zeros((num_val, num_trn))

#training set
for i in range(num_trn):
    print(i)
    for j in range(num_trn):
        n_trn_x[i, j] = similarity(trn_x[i], landmarks[j])
                                   
#val set                                   
for i in range(num_val):
    print(i)
    for j in range(num_trn):
        n_val_x[i, j] = similarity(val_x[i], landmarks[j])

trn_x = n_trn_x
val_x = n_val_x
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
#Computation graph params
input_size = trn_x.shape[1]
output_size = num_label
wd_rate = 0.00001
l_r = 0.0002
num_epoch = 500
batch_size = 20
dp_rate = 0

print(input_size, output_size)

696 1


In [5]:
dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_filename = dataset.maps
labels = dataset.labels

print(trn_y[19])
print(val_y[19])

display = plotting.plot_matrix(trn_x[19].reshape(48, 48), vmin=-1.0, vmax=1.0, colorbar=True, labels=labels[1:], figure = (10, 10))
plotting.show()

display = plotting.plot_matrix(val_x[19].reshape(48, 48), vmin=-1.0, vmax=1.0, colorbar=True, labels=labels[1:], figure = (10, 10))
plotting.show()

plt.savefig('1.pdf')

[0.]
[0.]


ValueError: cannot reshape array of size 696 into shape (48,48)

### Build Computation Graph

In [6]:
def forward_prop(cur_layer, out_size, dp_rate, is_input):
    
    _, cur_size = [item.value for item in cur_layer.shape]
        
    var = 2.0 / (cur_size + out_size)

    stddev = math.sqrt(var)

    print("steddev is", stddev)
    print(out_size)

    with tf.name_scope('fc'):
        W_fc = tf.Variable(tf.truncated_normal(shape=[cur_size,out_size], stddev=stddev))
        b_fc = tf.Variable(tf.constant(0.0, shape=[out_size]))
        if is_input is False: 
            cur_layer = tf.nn.dropout(cur_layer, 1 - dp_rate)
        out_layer = tf.matmul(cur_layer, W_fc) + b_fc

    return out_layer

def activation(cur_layer, is_output=False):
    if is_output is True:
        return tf.nn.sigmoid(cur_layer)
    return tf.nn.leaky_relu(cur_layer)

def loss_function(yhat, y):
    '''
    '''
    # get the weight decay terms
    with tf.name_scope('wd_term'):
        wght_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='fc')
        wd_term = wd_rate * tf.stack([tf.nn.l2_loss(i) for i in wght_params])

    with tf.name_scope('crit_loss'):
        crit = tf.reduce_mean(tf.losses.hinge_loss(labels=y, logits=yhat)) + tf.reduce_sum(wd_term)
        
    return crit

def generate_dataset(x, y, shuffle=True):
    print(batch_size)
    with tf.name_scope('dataset'):
        dataset = tf.data.Dataset.from_tensor_slices((x, y))
        if shuffle: 
            dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)
        else:
            dataset = dataset.batch(batch_size)

    return dataset.make_initializable_iterator()

In [7]:
#Build the actual graph
tf.reset_default_graph()
x_place = tf.placeholder('float', [None, input_size], name='input_x')
y_place = tf.placeholder('float', [None,  output_size], name='target_y')
dp_place = tf.placeholder(tf.float32, name='dropout_value')
lr_place = tf.placeholder(tf.float32, name='learning_rate')

cur_layer = x_place

cur_layer = forward_prop(cur_layer, output_size, dp_place, True)
yhat_op = activation(cur_layer, True)

#Cross entropy loss
loss_op = loss_function(yhat_op, y_place)
tf.summary.scalar('Cross_Entropy_Loss', loss_op)

#adam
optimizer = tf.train.AdamOptimizer(lr_place)
train_op = optimizer.minimize(loss_op)

#Accuracy
correct_pred = tf.equal(y_place, tf.round(yhat_op))   
acc_op = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
tf.summary.scalar('Accuracy', acc_op)

#Iterators
trn_it = generate_dataset(trn_x, trn_y, shuffle=True)
val_it = generate_dataset(val_x, val_y, shuffle=False)

steddev is 0.05356715840055802
1
20
20


In [8]:
#Actual train loop
if 'session' in locals() and session is not None:
    print('Close interactive session', flush=True)
    session.close()


sess = tf.Session()

# get the summary ready
merged = tf.summary.merge_all()
trn_writer = tf.summary.FileWriter("./log", sess.graph)

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

# tracing the loss along the way
meta_info = {}
meta_info['trn_loss'] = []
meta_info['trn_acc'] = []
meta_info['val_loss'] = []
meta_info['val_acc'] = []
meta_info['tst_loss'] = []
meta_info['tst_acc'] = []
iteration = 0

start = time.time()

for ep in range(num_epoch):
    sess.run(trn_it.initializer)
    sess.run(val_it.initializer)

    trn_next_batch = trn_it.get_next()

    avg_trn_loss = 0
    avg_trn_acc = 0
    
    n_lr = l_r
    print('TRN:', flush=True)
    while True: # loop to iterate over training set
        try: 
            x, y = sess.run(trn_next_batch)
        except: 
            break
        print('+', end='', flush=True)
        iteration += 1
        
        this_batch_size = x.shape[0]


        _, loss_i, acc_i, summary_i, _yhat, _pred = sess.run([train_op, loss_op, acc_op, merged, yhat_op, correct_pred], 
                                                   feed_dict={x_place:x, y_place:y, dp_place:dp_rate, lr_place:n_lr})

        trn_writer.add_summary(summary_i, iteration)
        avg_trn_loss = avg_trn_loss + loss_i*this_batch_size
        avg_trn_acc = avg_trn_acc + acc_i*this_batch_size
        
        '''print(_yhat, flush=True)
        print(y, flush=True)
        print(_pred, flush=True)
        print(acc_i, flush=True)'''
    
    avg_trn_loss = avg_trn_loss/num_trn
    avg_trn_acc = avg_trn_acc/num_trn
    
    #Validation
    avg_val_loss = 0
    avg_val_acc = 0
    print('VAL:', flush=True)
    # evaluate the loss and accuracy
    pred_and_gt = {}
    pred_and_gt['x'] = []
    pred_and_gt['y'] = []
    pred_and_gt['yhat'] = []
    
    val_next_batch = val_it.get_next()

    while True:
        try: x, y = sess.run(val_next_batch)
        except: break
        print('+', end='', flush=True)
        
        this_batch_size = x.shape[0]
        # compute the loss
        _yhat, _loss, _acc = sess.run([yhat_op, loss_op, acc_op], 
                                   feed_dict={x_place:x, y_place:y, dp_place:0.0})

        # keep track of the loss/pred_and_gt
        pred_and_gt['x'].append(x)
        pred_and_gt['y'].append(y)
        pred_and_gt['yhat'].append(_yhat)
        avg_val_loss = avg_val_loss + _loss*this_batch_size
        avg_val_acc = avg_val_acc + _acc*this_batch_size
    
    avg_val_loss = avg_val_loss/num_val
    avg_val_acc = avg_val_acc/num_val

    print('', flush=True)

    # concatenate the list to numpy array
    for k, v in pred_and_gt.items():
        pred_and_gt[k] = np.concatenate(v)
    
    
    print('\nEp.%05d - TRN: loss %.2f VAL: loss %.2f' % (ep, avg_trn_loss, avg_val_loss), flush=True)
    print('\nEp.%05d - TRN: acc %.2f VAL: acc %.2f' % (ep, avg_trn_acc, avg_val_acc), flush=True)
        
        
    # save learning meta information
    meta_info['trn_loss'].append(avg_trn_loss)
    meta_info['trn_acc'].append(avg_trn_acc)
    meta_info['val_loss'].append(avg_val_loss)
    meta_info['val_acc'].append(avg_val_acc)
    
print("Done")
end = time.time()
print('Time elapsed: %.2f seconds' % (end - start)) 
pkl.dump(meta_info, open(os.path.join('.', 'result.p'), 'wb'))
saver.save(sess, "./model/model")
sess.close()

TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00000 - TRN: loss 1.04 VAL: loss 1.01

Ep.00000 - TRN: acc 0.48 VAL: acc 0.47
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00001 - TRN: loss 1.03 VAL: loss 1.01

Ep.00001 - TRN: acc 0.50 VAL: acc 0.50
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00002 - TRN: loss 1.03 VAL: loss 1.01

Ep.00002 - TRN: acc 0.54 VAL: acc 0.51
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00003 - TRN: loss 1.03 VAL: loss 1.00

Ep.00003 - TRN: acc 0.57 VAL: acc 0.56
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00004 - TRN: loss 1.02 VAL: loss 1.00

Ep.00004 - TRN: acc 0.62 VAL: acc 0.59
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00005 - TRN: loss 1.02 VAL: loss 1.00

Ep.00005 - TRN: acc 0.67 VAL: acc 0.68
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00006 - TRN: loss 1.02 VAL: loss 1.00

Ep.00006 - TRN: acc 0.75 VAL: acc 0.68
TRN:
+++++++++++++++++++++++++++++++++++V

TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00060 - TRN: loss 0.86 VAL: loss 0.88

Ep.00060 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00061 - TRN: loss 0.86 VAL: loss 0.88

Ep.00061 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00062 - TRN: loss 0.86 VAL: loss 0.88

Ep.00062 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00063 - TRN: loss 0.86 VAL: loss 0.88

Ep.00063 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00064 - TRN: loss 0.86 VAL: loss 0.88

Ep.00064 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00065 - TRN: loss 0.85 VAL: loss 0.88

Ep.00065 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00066 - TRN: loss 0.85 VAL: loss 0.87

Ep.00066 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++V

TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00120 - TRN: loss 0.77 VAL: loss 0.82

Ep.00120 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00121 - TRN: loss 0.77 VAL: loss 0.82

Ep.00121 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00122 - TRN: loss 0.77 VAL: loss 0.81

Ep.00122 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00123 - TRN: loss 0.77 VAL: loss 0.81

Ep.00123 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00124 - TRN: loss 0.77 VAL: loss 0.81

Ep.00124 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00125 - TRN: loss 0.77 VAL: loss 0.81

Ep.00125 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00126 - TRN: loss 0.77 VAL: loss 0.81

Ep.00126 - TRN: acc 0.94 VAL: acc 0.81
TRN:
+++++++++++++++++++++++++++++++++++V

TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00180 - TRN: loss 0.72 VAL: loss 0.78

Ep.00180 - TRN: acc 0.96 VAL: acc 0.84
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00181 - TRN: loss 0.72 VAL: loss 0.78

Ep.00181 - TRN: acc 0.96 VAL: acc 0.85
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00182 - TRN: loss 0.72 VAL: loss 0.77

Ep.00182 - TRN: acc 0.96 VAL: acc 0.85
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00183 - TRN: loss 0.72 VAL: loss 0.77

Ep.00183 - TRN: acc 0.96 VAL: acc 0.85
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00184 - TRN: loss 0.72 VAL: loss 0.77

Ep.00184 - TRN: acc 0.96 VAL: acc 0.85
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00185 - TRN: loss 0.72 VAL: loss 0.77

Ep.00185 - TRN: acc 0.96 VAL: acc 0.85
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00186 - TRN: loss 0.72 VAL: loss 0.77

Ep.00186 - TRN: acc 0.96 VAL: acc 0.85
TRN:
+++++++++++++++++++++++++++++++++++V

TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00240 - TRN: loss 0.69 VAL: loss 0.75

Ep.00240 - TRN: acc 0.99 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00241 - TRN: loss 0.69 VAL: loss 0.75

Ep.00241 - TRN: acc 0.99 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00242 - TRN: loss 0.69 VAL: loss 0.75

Ep.00242 - TRN: acc 0.99 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00243 - TRN: loss 0.69 VAL: loss 0.75

Ep.00243 - TRN: acc 0.99 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00244 - TRN: loss 0.69 VAL: loss 0.75

Ep.00244 - TRN: acc 0.99 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00245 - TRN: loss 0.68 VAL: loss 0.75

Ep.00245 - TRN: acc 0.99 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00246 - TRN: loss 0.68 VAL: loss 0.75

Ep.00246 - TRN: acc 0.99 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++V

TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00300 - TRN: loss 0.66 VAL: loss 0.73

Ep.00300 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00301 - TRN: loss 0.66 VAL: loss 0.73

Ep.00301 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00302 - TRN: loss 0.66 VAL: loss 0.73

Ep.00302 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00303 - TRN: loss 0.66 VAL: loss 0.73

Ep.00303 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00304 - TRN: loss 0.66 VAL: loss 0.73

Ep.00304 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00305 - TRN: loss 0.66 VAL: loss 0.73

Ep.00305 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00306 - TRN: loss 0.66 VAL: loss 0.73

Ep.00306 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++V

TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00360 - TRN: loss 0.65 VAL: loss 0.71

Ep.00360 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00361 - TRN: loss 0.65 VAL: loss 0.71

Ep.00361 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00362 - TRN: loss 0.65 VAL: loss 0.71

Ep.00362 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00363 - TRN: loss 0.65 VAL: loss 0.71

Ep.00363 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00364 - TRN: loss 0.65 VAL: loss 0.71

Ep.00364 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00365 - TRN: loss 0.65 VAL: loss 0.71

Ep.00365 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00366 - TRN: loss 0.65 VAL: loss 0.71

Ep.00366 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++V

TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00420 - TRN: loss 0.63 VAL: loss 0.70

Ep.00420 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00421 - TRN: loss 0.63 VAL: loss 0.70

Ep.00421 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00422 - TRN: loss 0.63 VAL: loss 0.70

Ep.00422 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00423 - TRN: loss 0.63 VAL: loss 0.70

Ep.00423 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00424 - TRN: loss 0.63 VAL: loss 0.70

Ep.00424 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00425 - TRN: loss 0.63 VAL: loss 0.70

Ep.00425 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00426 - TRN: loss 0.63 VAL: loss 0.70

Ep.00426 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++V

TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00480 - TRN: loss 0.62 VAL: loss 0.69

Ep.00480 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00481 - TRN: loss 0.62 VAL: loss 0.69

Ep.00481 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00482 - TRN: loss 0.62 VAL: loss 0.69

Ep.00482 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00483 - TRN: loss 0.62 VAL: loss 0.69

Ep.00483 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00484 - TRN: loss 0.62 VAL: loss 0.69

Ep.00484 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00485 - TRN: loss 0.62 VAL: loss 0.69

Ep.00485 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++VAL:
+++++++++

Ep.00486 - TRN: loss 0.62 VAL: loss 0.69

Ep.00486 - TRN: acc 1.00 VAL: acc 0.92
TRN:
+++++++++++++++++++++++++++++++++++V

In [43]:
tf.reset_default_graph()

imported_graph = tf.train.import_meta_graph('./model/model.meta')

for tensor in tf.get_default_graph().get_operations():
    print (tensor.name)

test_x = np.zeros((1, 2304))
test_x[0, :] = val_x[50]

with tf.Session() as sess:
    imported_graph.restore(sess, './model/model')
    yhat = sess.run(['Sigmoid:0'], feed_dict={'input_x:0':test_x, 'dropout_value:0':0.0})
    print(yhat)

print(val_y[50])

input_x
target_y
dropout_value
learning_rate
fc/truncated_normal/shape
fc/truncated_normal/mean
fc/truncated_normal/stddev
fc/truncated_normal/TruncatedNormal
fc/truncated_normal/mul
fc/truncated_normal
fc/Variable
fc/Variable/Assign
fc/Variable/read
fc/Const
fc/Variable_1
fc/Variable_1/Assign
fc/Variable_1/read
fc/MatMul
fc/add
Sigmoid
wd_term/L2Loss
wd_term/L2Loss_1
wd_term/stack
wd_term/mul/x
wd_term/mul
crit_loss/hinge_loss/ones_like/Shape
crit_loss/hinge_loss/ones_like/Const
crit_loss/hinge_loss/ones_like
crit_loss/hinge_loss/mul/x
crit_loss/hinge_loss/mul
crit_loss/hinge_loss/Sub
crit_loss/hinge_loss/Mul
crit_loss/hinge_loss/Sub_1
crit_loss/hinge_loss/Relu
crit_loss/hinge_loss/assert_broadcastable/weights
crit_loss/hinge_loss/assert_broadcastable/weights/shape
crit_loss/hinge_loss/assert_broadcastable/weights/rank
crit_loss/hinge_loss/assert_broadcastable/values/shape
crit_loss/hinge_loss/assert_broadcastable/values/rank
crit_loss/hinge_loss/assert_broadcastable/static_scalar_che

[array([[0.00998444]], dtype=float32)]
[0.]
